In [19]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_validate
from sklearn.linear_model import SGDRegressor

from preprocess import *

In [2]:
df = pd.read_csv('/content/drive/MyDrive/Le Wagon/Project Data/clean_data.csv')

In [3]:
df.head()

,TRAIN_SERVICE_CODE_AFFECTED,SERVICE_GROUP_CODE_AFFECTED,ENGLISH_DAY_TYPE,APP_TIMETABLE_FLAG_AFF,UNIT_CLASS_AFFECTED,INCIDENT_REASON,PERFORMANCE_EVENT_CODE,PFPI_MINUTES,Lat_OR,Lon_OR,...,ORIG_MINUTE_SIN,ORIG_MINUTE_COS,DEST_MONTH_SIN,DEST_MONTH_COS,DEST_DAY_SIN,DEST_DAY_COS,DEST_HOUR_SIN,DEST_HOUR_COS,DEST_MINUTE_SIN,DEST_MINUTE_COS
0,22214000,EK01,SA,Y,375.0,MD,M,9.0,51.54343,-0.02447,...,-1.000000,-1.836970e-16,0.951057,0.309017,7.431448e-01,0.669131,0.951057,0.309017,-0.978148,0.207912
1,22214000,EK01,WD,Y,378.0,MD,M,12.0,51.54343,-0.02447,...,-0.587785,-8.090170e-01,0.951057,0.309017,9.135455e-01,-0.406737,0.866025,0.500000,-0.809017,-0.587785
2,22214000,EK01,WD,Y,378.0,X8,M,8.0,51.54343,-0.02447,...,0.978148,-2.079117e-01,0.104528,0.994522,5.665539e-16,-1.000000,0.809017,-0.587785,0.913545,-0.406737
3,22214000,EK01,SA,Y,375.0,TG,M,10.0,51.54343,-0.02447,...,-0.309017,-9.510565e-01,0.913545,0.406737,5.877853e-01,-0.809017,0.913545,-0.406737,-0.207912,-0.978148
4,22214000,EK01,SA,Y,375.0,TG,M,11.0,51.54343,-0.02447,...,-0.669131,7.431448e-01,0.913545,0.406737,5.877853e-01,-0.809017,0.913545,-0.406737,-0.500000,0.866025


In [4]:
pipe = preprocessing_pipe()

In [5]:
X = df.drop(columns=['PFPI_MINUTES'])
y = df['PFPI_MINUTES']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [7]:
pipe.fit(X_train)

Pipeline(steps=[('transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('minmaxscaler',
                                                  MinMaxScaler(),
                                                  ['Lat_OR', 'Lon_OR',
                                                   'Lat_DES', 'Lon_DES']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['ENGLISH_DAY_TYPE',
                                                   'SERVICE_GROUP_CODE_AFFECTED',
                                                   'INCIDENT_REASON',
                                                   'UNIT_CLASS_AFFECTED',
                                                   'TRAIN_SERVICE_CODE_AFFECTED',
                                                   'PERFORMANCE_EVENT_CODE',
                                                   'APP_TIMETABLE_FLAG_AFF'])]))])

In [8]:
X_train_processed = pipe.transform(X_train)
X_test_processed = pipe.transform(X_test)

In [14]:
model = SGDRegressor(loss='squared_error', learning_rate='optimal', alpha=0.001, max_iter=1000)

In [17]:
model.fit(X_train_processed, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


SGDRegressor(alpha=0.001, learning_rate='optimal')

In [26]:
cv_results = cross_validate(model, X_train_processed, y_train, cv=5, scoring='neg_mean_absolute_error', verbose=1, n_jobs=4)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed: 25.2min finished


In [29]:
cv_results

{'fit_time': array([1135.07577062, 1137.9203887 , 1133.94998837, 1137.58496666,
         374.22410941]),
 'score_time': array([0.10763788, 0.0392127 , 0.08862901, 0.03964949, 0.03295684]),
 'test_score': array([-4.66250395e+07, -1.90101415e+08, -4.49742551e+07, -9.56794718e+07,
        -2.63984481e+08])}

In [34]:
y_pred = model.predict(X_test_processed)

In [35]:
model.score(X_train_processed, y_train)

-101731383787400.2